# LangChain Chatbot

In [9]:
import os

import pandas as pd
pd.set_option('display.max_colwidth', None)

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [10]:
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredHTMLLoader
from langchain.document_loaders import UnstructuredPDFLoader
from langchain import PromptTemplate

In [11]:
import langchain
langchain.debug = False

## load data and setup vector store

In [12]:
documents = []
for file in os.listdir('data/test_documents'):
    if file.endswith('.pdf'):
        pdf_path = './data/test_documents/' + file
        print(f'Loading {pdf_path}')
        loader = UnstructuredPDFLoader(pdf_path)
        documents.extend(loader.load())
    elif file.endswith('.html'):
        doc_path = './data/test_documents/' + file
        print(f'Loading {doc_path}')
        loader = UnstructuredHTMLLoader(doc_path)
        documents.extend(loader.load())

Loading ./data/test_documents_embedding_test/Extension of Deadlines Master Business Infromatics.html


In [13]:
# split documents into text chunks

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunked_documents = text_splitter.split_documents(documents)

In [14]:
# create chroma vector db with OpenAIEmbeddings

vectordb = Chroma.from_documents(
  chunked_documents,
  embedding=OpenAIEmbeddings(),
  persist_directory='./storage_langchain'
)
vectordb.persist()

### create QA chain

In [8]:
# Create Prompt
template = """

Use the following pieces of context to answer the question at the end.

Execute these steps:
1 - list the context
2 - focus on words like "optional" or "can" for your answer
3 - answer the question. Do not use information outside of the context to answer the question.

Your answer should have this format:

context:
answer:

------------------------
Context: {context}

Question: {question}

"""

custom_prompt = PromptTemplate.from_template(template)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model='gpt-3.5-turbo', temperature=0),
    retriever=vectordb.as_retriever(search_kwargs={'k': 5}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": custom_prompt}
)

# Tests
### read questions and answer

In [ ]:
df_questions = pd.read_csv('TestQuestions.csv', delimiter=";", names=["Question", "Response"] )
questions = df_questions["Question"]

responses = []
counter = 0
for q in questions:
    print(f'q{counter} start')
    r = qa_chain({'query': q})['result']
    responses.append((q, r))
    print(f'q{counter} end')
    counter += 1

df_responses = pd.DataFrame(responses, columns=["Question", "Response"])

In [ ]:
df_responses

### save responses

In [ ]:
# df_responses.to_csv("test_responses_by_langchain_.csv", sep=";")